# It's a Small World After All? : Degrees of Separation at W&M #

*“I feel like there are only 2 degrees of separation here”*

Sound familiar? It’s a phrase that often circulates the campus of William & Mary when students discover a close friend from one organization knows another friend from an entirely different avenue. At a college that thrives on community, I wanted to take a deeper look at the mutual connections on our campus. 

According to Stanley Milgram’s research, the world is only separated by six degrees of separation, meaning that two people can be linked by six connections. The Small World Project, conducted by Columbia University, aims to see if Milgram’s theory still stands in an ever-connected world. While Milgram’s study asked participants to send postcards, more recent users were asked to connect to a “target” person through email without directly contacting them. This project has found an average number of links to be six, which still supports Milgram’s theory. 

A [2012 study](https://arxiv.org/abs/1111.4570), however, suggests that the degrees are shrinking with the help of social media. Of course, globalization and the increase of technology has increased our connectedness, sometimes at the expense of closeness. At a small liberal arts college with varied academic and extracurricular interests, it is easy to wonder if our community is more connected than others.

## Data Cleaning ##

When I collected data, I asked for participants’ top 5-6 connections to exclude extraneous, loose connections, though they may be useful in a larger study. I asked participants to share the survey with the friends they listed in their response, in an effort to use snowball sampling. This method worked, to an extent, as I received 82 responses. I asked participants for their year, major and minor, and activities they were involved with. When asking for their top connections, I asked for the friend's year and how they knew eachother, which was at times multivaried.

Before processing the data with code, I cleaned the reponses:
* proper case for all strings, and title case for names
* removed concentrations for clarity
* standardized majors and activities among all responses (ie. Comp Sci -> Computer Science)
* used acronyms where widely accepted/well known (ie. Environmental & Sustainability Majors -> ENSP)
* removed extraneous/invalid responses (ie. "Just kinda being around I guess")

I also anonymized the names using [Faker](https://faker.readthedocs.io/en/master/), which maps the real names to fake ones. While I could have generated IDs or other alphanumerical representations of participants, I chose fake names to maintain a "human" element to the data.

In [3]:
import pandas as pd 
import numpy as np 
import random as rand

In [7]:
data = pd.read_csv('anonymized_data.csv')
data

,Unnamed: 0,Name,Year,Major,Second major/minor,Activities,Person1,Connection1,Year1,Person2,...,Year3,Person4,Connection4,Year4,Person5,Connection5,Year5,Person6,Connection6,Year6
0,0,Wayne Lee,2024,Anthropology,AMES,"Cypress Ultimate Frisbee A, Fitness Instructor...",Theresa Green DDS,Roommate,2024,Margaret Moore,...,2024,Daniel Rodriguez,class,2024,Justin Bentley,Cypress Ultimate Frisbee,2024,Christopher Stevens,before college,2024.0
1,1,Theresa Green DDS,2024,Computer Science,Music,"W&M Choir, APO, geoLab, Sinfonicron",Wayne Lee,Roommate,2024,Linda Martinez,...,2024,Megan Ramirez,Choir,2025,Johnny Glass,Choir,2026,Kristy Morales,Sinfonicron,2024.0
2,2,Jason Clarke,2024,Biology,NaN,"Bhangra, Food for All, Mortar Board, SASA",Theresa Green DDS,Dorm,2024,Linda Martinez,...,2024,Jennifer Alvarez,class,2024,Kelsey Dixon,SASA,2024,Alyssa Tucker,Bhangra,2023.0
3,3,Dawn Stephens,2024,Music,GSWS,"Pep Band, Tour Guide, Wind Ensemble, Wham Bam ...",Robin Anderson,Senior Interviewer,2024,Heather Clarke,...,2025,Matthew Watkins,Wind Ensemble,2025,Teresa Morris,Tour Guide,2024,Kevin Flynn,Dorm,2024.0
4,4,Linda Martinez,2024,Government,NaN,"Club Field Hockey, Botany Club, Tour Guide, PI...",Melissa Clark,before college,2024,George Arroyo,...,2024,Derrick Mccoy,Study abroad,2023,Andrea Jenkins,class,2024,Christopher Perez,Dorm,2024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,Gregory Adams,2024,Neuroscience,Biochemistry,"Bestman Neuroscience Lab, The Flat Hat, APO",Mariah Taylor,Roommate,2024,Roy Rivera,...,2024,Taylor Miller,The Flat Hat,2024,Sue Schmidt,APO,2024,Daniel Johnson,Research,2024.0
78,78,Jason Clarke,2024,Biology,Data Science,"Bhangra, Food for All, Mortar Board, SASA",Joseph Phelps,Bhangra,2025,Christopher Bass,...,2024,Ricky Benton,class,2024,Andrew Brandt,Bhangra,2025,Jennifer Anderson,class,2025.0
79,79,Jennifer Anderson,2025,Neuroscience,Public Health,"Sorority, Peer Advising, STEM Panel, EMT",Robert Barber,Dorm,2025,Joseph Austin,...,2025,Anthony Dixon,Sorority,2025,Kelsey Dixon,Sorority,2024,NaN,NaN,NaN
80,80,Jennifer Alvarez,2024,Biology,NaN,"Premed Club, IM Football",Angie Wright,Roommate,2024,Christopher Bass,...,2024,Jason Clarke,Friend,2024,Brenda Brown,Friend,2024,Theresa Green DDS,Friend,2024.0


Now that the data has been loaded, we will melt the wide-form data into individual "to-from" connections along with the information indicating how they are connected. Since the 6th connection was optional, all empty connections were dropped.

In [8]:
connect = pd.DataFrame()
for i in range(1, 7):
    subset = ["Name", "Person" + str(i), "Connection" + str(i), ]
    sub_data = data[subset]
    sub_data.columns = ["name1", "name2", "connection"]
    connect = pd.concat([connect, sub_data], axis=0)

connect = connect.dropna(thresh=2)
connect

,name1,name2,connection
0,Wayne Lee,Theresa Green DDS,Roommate
1,Theresa Green DDS,Wayne Lee,Roommate
2,Jason Clarke,Theresa Green DDS,Dorm
3,Dawn Stephens,Robin Anderson,Senior Interviewer
4,Linda Martinez,Melissa Clark,before college
...,...,...,...
76,Shannon Johnston,Pamela Martinez,class
77,Gregory Adams,Daniel Johnson,Research
78,Jason Clarke,Jennifer Anderson,class
80,Jennifer Alvarez,Theresa Green DDS,Friend


Now that we've melted down all our data to "to-from" connections, we are going to start building a profile for each person included in the data to include in our network graph. Similar to anonymizing the data, I want to keep a "human" element to each node. This data will also be used to categorize based on year, major, or activities in later plots.

The people who were mentioned as connections but did not participate themselves would only have their name and year as personal attributes, but they are also included.

In [9]:
names = pd.concat([connect['name1'], connect['name2']])
names = names.unique()
personal = data[['Name', 'Year', 'Major', "Second major/minor", 'Activities', ]]

In [10]:
# add people who were named as connections along with their year
connection_personal = pd.DataFrame()
for i in range(1, 7):
    cols = ["Person" + str(i), "Year" + str(i)]
    sub_data = data[cols]
    sub_data.columns = ['Name', 'Year']
    connection_personal = pd.concat([connection_personal, sub_data], axis=0)

connection_personal['Year'] = connection_personal['Year'].astype(str).apply(lambda x: x.replace('.0',''))
connection_personal.dropna(inplace=True)
connection_personal.drop_duplicates(inplace=True)

,Name,Year
0,Theresa Green DDS,2024
1,Wayne Lee,2024
3,Robin Anderson,2024
4,Melissa Clark,2024
5,Megan Ramirez,2025
...,...,...
75,Susan Griffin,2026
76,Pamela Martinez,2024
77,Daniel Johnson,2024
78,Jennifer Anderson,2025


In [12]:
# if name does not already exist in dataframe (ie. they did not fill out survey themselves), concatenate to personal info data
# other unknown values are Nan
for i, row in connection_personal.iterrows():
    if row['Name'] not in personal['Name'].values:
        add = pd.DataFrame([[row['Name'], row['Year']]], columns=['Name', 'Year'])
        personal = pd.concat([personal, add], axis=0)

personal

,Name,Year,Major,Second major/minor,Activities
0,Wayne Lee,2024,Anthropology,AMES,"Cypress Ultimate Frisbee A, Fitness Instructor..."
1,Theresa Green DDS,2024,Computer Science,Music,"W&M Choir, APO, geoLab, Sinfonicron"
2,Jason Clarke,2024,Biology,NaN,"Bhangra, Food for All, Mortar Board, SASA"
3,Dawn Stephens,2024,Music,GSWS,"Pep Band, Tour Guide, Wind Ensemble, Wham Bam ..."
4,Linda Martinez,2024,Government,NaN,"Club Field Hockey, Botany Club, Tour Guide, PI..."
...,...,...,...,...,...
0,Stephanie Pineda,2025,NaN,NaN,NaN
0,Heather Summers,2025,NaN,NaN,NaN
0,Susan Griffin,2026,NaN,NaN,NaN
0,Pamela Martinez,2024,NaN,NaN,NaN


## Categorize Connections ##
**How are people at W&M commonly linked? Are there patterns among how people are connected? Are there common pipelines between certain groups?**

By categorizing the way in which people are connected, I hope to explore these questions.

I collected all the information listed when asked how participants knew a given person. In gathering this data, I grouped the collection into 11 categories. I later combined `music` and `performing` for simplicity.

In [13]:
Greek_life = ['APO', 'Alpha Chi Omega', 'Fraternity', 'Gamma Phi Beta', 'Kappa Delta', 'Kappa Delta Rho', 'Gamma Phi', 'Nu Kappa Epsilon', 'Phi Mu', 'Pi Beta Phi', 'Sorority', 'greek life']
Music = ['Acapella', 'Choir', 'Christopher Wren Singers', 'Crim Dell Criers', 'Music', 'Pep Band', 'The Stairwells', 'WMSO', 'Wind Ensemble', 'Wham Bam Big Band']
Performing = ['Ballroom', 'Bhangra', 'Orchesis Dance Company', 'Pointe Blank', 'SITD', 'Sinfonicron', 'Swing Club', 'Theater', 'Trippin on Brix']
Performing += Music
Cultural_Religious = ['African Cultural Society', 'FASA', 'JCA', 'SASA', 'RUF', 'Hillel']
Other_org = ['AMP', 'Club', 'Classics Club', 'D&D', 'Veggie Society', 'Dining Sustainability', 'Mock Trial', 'Nerf Club', 'The Flat Hat', 'IRC', 'Young Democrats', 'WCWM Radio']
Sport = ['Archery Club', 'Club Field Hockey', 'Club Lacrosse', 'Club Sailing', 'Club Volleyball', 'Cross Country', 'Cypress Ultimate Frisbee', 'Shotokan Karate', 'Ultimate Frisbee', 'Woolly Mammoths Ultimate Frisbee']
Academic = ['Courageous Leadership & Authentic Excellence Fellowship', 'Major class', 'Research', 'Scholarship program', 'Study abroad', 'Washington Center', 'class']
Relationships = ['girlfriend', 'boyfriend', 'Friend', 'Mutual friend', 'Tinder']
Living = ['Dorm', 'Roommate']
Work = ['Residence Life', 'OA', 'Senior Interviewer', 'Student Assembly', 'Fitness Instructor', 'Tour Guide', 'Work']
before_college = ['before college']

In [14]:
cat_dict = {'Greek Life': Greek_life, "Performing": Performing, "Cultural/Religious Org": Cultural_Religious, 'Other Org': Other_org, \
            "Sport": Sport, "Academic": Academic, "Relationships": Relationships, "Living": Living, "Work": Work, "Before College": before_college}

Using the categories I defined, I added a **category** attribute to the `connect` dataframe, which will later be used for the network graph.

*Note: several responses listed many ways they knew a person. I chose to use the first listed activity for the sake of this project. It is likely that the primary way in which two people are connected was listed first.*

In [15]:
connect['category'] = " "
for i, row in connect.iterrows():
    link = row['connection'].split(", ")[0] # use the first listed connection to categorize the link
    for cat in cat_dict:
        if link in cat_dict[cat]:
            row['category'] = cat
            break
connect

,name1,name2,connection,category
0,Wayne Lee,Theresa Green DDS,Roommate,Living
1,Theresa Green DDS,Wayne Lee,Roommate,Living
2,Jason Clarke,Theresa Green DDS,Dorm,Living
3,Dawn Stephens,Robin Anderson,Senior Interviewer,Work
4,Linda Martinez,Melissa Clark,before college,Before College
...,...,...,...,...
76,Shannon Johnston,Pamela Martinez,class,Academic
77,Gregory Adams,Daniel Johnson,Research,Academic
78,Jason Clarke,Jennifer Anderson,class,Academic
80,Jennifer Alvarez,Theresa Green DDS,Friend,Relationships


## Plotting the Network ##
I chose to use [pyvis](https://pyvis.readthedocs.io/en/latest/documentation.html) to plot the network, which is built on [visjs](https://github.com/visjs), allowing users to widely customize the display of their visualizations.

In [ ]:
! pip install pyvis
! pip install networkx

In [16]:
import matplotlib.pyplot as plt
from pyvis.network import Network
import networkx as nx

In [17]:
palette = {'purple': '#7f3c8d', 'green': '#11a579', "blue": '#3969ac', 'yellow': '#f2b601', 'lightgreen': '#80ba5a',
                 'orange': '#e68210', 'teal': '#008695', 'pink': '#cf1c90', 'salmon': '#f97b72', 'grey': '#a5aa99'}

# mapping each connection category to a color, to be used to color edges in the graph
color_dict = {'Greek Life': palette['pink'], "Performing": palette['blue'], "Cultural/Religious Org": palette['orange'], 'Other Org': palette['yellow'], 
            "Sport": palette['green'], "Academic": palette['purple'], "Relationships": palette['teal'], "Living": palette['salmon'], "Work": palette['grey'], "Before College": palette['lightgreen']}

In [18]:
# used to format the information listed when hovering over nodes, which includes [fake]name, year, major/minor, and activities

def format_hover(person_info):
    title = str(person_info['Name'].values[0]) + " (" + str(person_info['Year'].values[0]) + ")\n"
    
    if not pd.isna(person_info['Major'].values[0]):
        title += str(person_info['Major'].values[0])
    
    if not pd.isna(person_info['Second major/minor'].values[0]):
        title += ", " + str(person_info['Second major/minor'].values[0])
    
    if not pd.isna(person_info['Activities'].values[0]):
        title += "\nActivities: " + str(person_info['Activities'].values[0])
    return title

In [21]:
NODE_SIZE = 10
COLOR_STYLE = [{
                  'border': 'grey',
                  'background': '#454B53',
                  'highlight': {
                    'border': 'white',
                    'background': '#454B53',
                  }}]
OPTIONS = """const options = {
  "nodes":{
    "borderWidth": 1,
    "borderWidthSelected": 2,
    "chosen": true,
    "color": {
      "border": "black",
      "background": "white",
      "highlight": {
        "border": "gray",
        "background": "gray"
      },
      "hover": {
        "border": "#bfbca4",
        "background": "#f0ecd3"
      }
    },
    "opacity": 1,
    "font": {
      "color": "#343434",
      "size": 14,
      "face": "arial",
      "background": "none",
      "strokeWidth": 0,
      "strokeColor": "#ffffff",
      "align": "center'",
      "multi": false,
      "vadjust": 0,
      "bold": {
        "color": "#343434",
        "size": 14,
        "face": "arial",
        "vadjust": 0,
        "mod": "bold"
      },
      "mono": {
        "color": "#343434",
        "size": 15,
        "face": "courier new",
        "vadjust": 2
      }
    }
  }
}
"""

To add people (nodes) and connections (edges) to the graph, I iterrated through the `connect` dataframe. For each entry, I formatted the personal information. I also selected the edge color based on the `category` attribute. 

*The graph will be generated when run in a web-based format of ipynb (ie. [Google Colab](https://colab.google/), jupyter notebook). It will also save as a html file that you can view in a browser, which I recommend.*

In [ ]:
from IPython.display import Image
Image(filename='/content/legend2.PNG', width = '600', height='400')

In [22]:
graph = Network(filter_menu=False, notebook=True)

# add legend to plot
graph.add_node(0, shape='image', image ="/content/legend2.png", size = 300)

for i, row in connect.iterrows():

    person_info1 = personal.loc[personal['Name'] == row['name1']]
    person_info2 = personal.loc[personal['Name'] == row['name2']]
    title1 = format_hover(person_info1)
    title2 = format_hover(person_info2)
    
    color = color_dict[row['category']] #pick edge color based on connection category
   
    graph.add_node(row['name1'], title = title1, size = NODE_SIZE, color = COLOR_STYLE)
    graph.add_node(row['name2'], title = title2, size = NODE_SIZE, color = COLOR_STYLE)
    graph.add_edge(row['name1'], row['name2'], color=color, weight = 20)

graph.set_options(OPTIONS)
graph.show("all_connections.html") #may take a minute to load in colab files

### Degrees of Separation ###
Though the network shows the interconnectedness of W&M, it can be difficult to study the degrees of separation overall. In looking closer at the degrees of separation, I used [bidirectional BFS (breadth-first search)](https://medium.com/@zdf2424/discovering-the-power-of-bidirectional-bfs-a-more-efficient-pathfinding-algorithm-72566f07d1bd) to find the smallest distance between two nodes. By running the algorithm from the start and target nodes simultaneously, the search is more efficient.

In [ ]:
! pip install plotly

In [23]:
import plotly.figure_factory as ff

In [24]:
adj_list = graph.get_adj_list() #get list of nodes with neighbors

def bidirectionalBFS(startNode, endNode):
    queueStart = [startNode]
    tmpQS = []
    queueEnd = [endNode]
    tmpQE = []
    visitStart = set()
    visitEnd = set()
    visitStart.add(startNode)
    visitEnd.add(endNode)

    #loop until both queues are empty or until connection found
    count = 1
    while (len(queueStart) > 0 and len(queueEnd) > 0):
        
        while len(queueStart) > 0:
            currStart = queueStart.pop(0)

            for adj in adj_list[currStart]: 
                if not visitStart.issuperset({adj}):
                    tmpQS.append(adj)
                    visitStart.add(adj)
                if visitEnd.issuperset({adj}):
                    return count
                
        queueStart = tmpQS.copy()
        tmpQS.clear()
        count += 1
            
        while len(queueEnd) > 0:
            currEnd = queueEnd.pop(0)
            for adj in adj_list[currEnd]: 
                if not visitEnd.issuperset({adj}):
                    tmpQE.append(adj)
                    visitEnd.add(adj)
                if visitStart.issuperset({adj}):
                    return count
                
        queueEnd = tmpQE.copy()
        tmpQE.clear()
        count += 1
            
    return 0 # if no connection found

To show the degrees of separation, I randomly chose five nodes from the network and found the degrees of separation between each of the five targets and all the remaining nodes.

In [26]:
targets = rand.choices(list(adj_list.keys()), k=5)

t1, t2, t3, t4, t5 = [], [], [], [], []
degrees = [t1, t2, t3, t4, t5]

for target, d in zip(targets, degrees):
    for key in adj_list:
        if key != target:
            degree = bidirectionalBFS(key, target)
            d.append(degree)

I chose to filter out nodes that did not connect to the target, as I believe this was likely due to a lack of data. In reality, there are likely additional connections not reflected in the network due to the way in which data was collected.

In [27]:
# filtering out node results that did not connect to targets
filtered_deg = []
for row in degrees:
    row = list(filter(lambda x: x != 0, row))
    filtered_deg.append(row)

In [29]:
LINE_COLORS = [palette['purple'], palette['yellow'], palette['green'], palette['blue'], palette['salmon']]

fig = ff.create_distplot(filtered_deg, show_hist=False, group_labels=targets, show_rug=False, colors= ['#7c6354', '#b49fcc', '#95d9c3', '#ead7d7', "#ffd6ba"])
fig.update_layout(title_text='<b>Degrees of Separation at W&M Match Global Average</b>', xaxis_title="Degrees of Separation (edges)", 
                    yaxis_title="Density", legend_title = "Targets", plot_bgcolor="white",
                    font=dict(
                        size = 14
                    ))
fig.update_xaxes(showline=True, gridcolor="lightgrey")
fig.update_yaxes(showline=True, gridcolor="lightgrey")

fig.show()

In [30]:
# run to save the plot to a file
fig.write_html("Degrees_of_Sep_distplot.html")

The distplot above reflects the univariate distribution of the data. Since the max of each curve is approximately 6, the degrees of separation for William & Mary can be said to reflect Milgram's theory. Although this small subset of the W&M population suggests that we are not any more connected than the global population, a larger and more comprehensive study could suggest otherwise. 

Nevertheless, it is still interesting to explore the hidden connections within the network graph. For example, Jennifer Alvarez and Jason Clarke know eachother from an academic experience. Jennifer knows Christopher Bass from living together while Jason knows them from a relationship (ie. significant other, mutual friend). Perhaps Jennifer is the mutual friend that connects Jason and Christopher, or perhaps they are friends in their own right. 

In another example, Jason Clarke knows Jennifer Anderson from an academic activity and Kelsey Dixon from a cultural or religious organization, but Jennifer and Kelsey know eachother from Greek life. Do Jason and Jennifer know that they are both connected to Kelsey through entirely different avenues?

*Note: due to the random nature of target selection, there is a chance that a node that is not connected to the central network was selected. In this case, the degrees of separation are limited to the number of nodes in the sub-network, which results in a drop-off of data in the distplot.*

In [34]:
print("Average Degrees of Separation for each Target: ")
for target, d in zip(targets, degrees):
    a = sum(d)/len(d)
    print(target + ": " + str(a))

Average Degrees of Separation for each Target: 
Andrea Williams: 4.064432989690721
Darren Simmons: 5.65979381443299
Beth Lopez: 0.028350515463917526
Cameron Hendricks: 4.889175257731959
Debbie Stevens: 4.969072164948454


The average degrees of separation for each target suggest that we are connected slightly closer than the global average. As stated previously, further study with a larger sample may help gain more information.

### Further Plotting ###
While there are many attributes we could explore further, we will look closer at class year. Using the existing network nodes, I changed the node color based on year and made the color of all edges grey. Though each respective year is well-connected, as expected, there are many multi-year connections beyond the central network.

For example, the cluster that includes Chad King, Alyssa Nash, and Megan Williams is varied in year but is well connected. While the classes of 2024 and 2025 are well connected within the center of the network, there are still significant clusters of the class of 2026 that are connected.

In [ ]:
Image(filename='/content/year_legend.PNG', width = '300', height='400')

In [36]:
year_color = {'2024': palette['teal'], '2025': palette['yellow'], '2026': palette['purple'], '2027': palette['salmon']}
graph.nodes[0]['image'] = "/content/year_legend.png"

for node in graph.nodes:
    if node['id'] != 0:
        yr_idx = node['title'].find("(")
        yr = node['title'][yr_idx+1: yr_idx+5]
        col = year_color.setdefault(yr, palette['grey'])
        node['color'] = col

for edge in graph.edges:
    edge['color'] = 'gray'

graph.show('connections_by_year.html')

## Closing remarks ##

Though this study did not reveal a closer network than the global average, a larger study within the W&M community may produce different results. With more data, we may gain more information regarding frequent pipelines of connections, especially suprising combinations of interests or activities.

While we looked at connections based on year, a deeper dive into the network based on major and academic connections may reveal correlations among subsets. A closer look at specific elements within connection categories may also reveal tight-knit communities.
